In [687]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
import math as m
import itertools

In [1018]:
def plot_3d(sides) -> None:
    fig = px.scatter_3d()
    for face, vals in sides.items():
        try:
            fig.add_trace(go.Scatter3d(x=vals[:, 0].A1, y=vals[:, 1].A1, z=vals[:, 2].A1, name=face))
        except AttributeError:
            fig.add_trace(go.Scatter3d(x=vals[:, 0], y=vals[:, 1], z=vals[:, 2], name=face))


    fig['layout']['scene']['aspectmode'] = "cube"
    fig['layout']['scene']['zaxis']['range'] = [-300, 300]
    fig['layout']['scene']['yaxis']['range'] = [-300, 300]
    fig['layout']['scene']['xaxis']['range'] = [-300, 300]
    fig['layout']['scene']['xaxis']['zerolinecolor'] = 'black'
    fig['layout']['scene']['yaxis']['zerolinecolor'] = 'black'
    fig['layout']['scene']['zaxis']['zerolinecolor'] = 'black'
    fig.show()


def rotate(A, axis, theta):
    if axis == 'x':
        T = np.matrix([[1, 0, 0],
                        [0, m.cos(theta), -m.sin(theta)],
                        [0, m.sin(theta), m.cos(theta)]])
    elif axis == 'y':
        T = np.matrix([[m.cos(theta), 0, m.sin(theta)],
                       [0, 1, 0],
                       [-m.sin(theta), 0, m.cos(theta)]])
    elif axis == 'z':
        T = np.matrix([[m.cos(theta), -m.sin(theta), 0],
                       [m.sin(theta), m.cos(theta), 0],
                       [0, 0, 1]])
    else:
        raise AssertionError(f'{axis=} not valid')
    return np.matmul(A, T).round(1)


def translate3d(A, dx, dy, dz):
    A = np.c_[A[:, :3], np.ones(A.shape[0])]
    T = np.matrix([[1, 0, 0, dx],
                   [0, 1, 0, dy],
                   [0, 0, 1, dz],
                   [0, 0, 0, 1]])
    return np.matmul(T, A.T).T[:, :3].round(1)


def vertices(*args):
    """Return the 4 vertices of one or several matrices."""
    ret = set()
    for A in args:
        xs, ys, zs = list(zip(*A))
        ret.add((min(xs), min(ys), min(zs)))
        ret.add((min(xs), min(ys), max(zs)))  
        ret.add((max(xs), min(ys), min(zs)))
        ret.add((max(xs), min(ys), max(zs)))  
        ret.add((max(xs), max(ys), min(zs))) 
        ret.add((max(xs), max(ys), max(zs))) 
        ret.add((min(xs), max(ys), min(zs)))
        ret.add((min(xs), max(ys), max(zs)))
    return tuple(ret)
    

def edge(*args, onto):
    """Return the pivot edge by which we need to offset"""
    for a in np.array(vertices(*args)):
        for b in np.array(vertices(onto)):
            if np.linalg.norm(b - a) == 1:
                return a
    raise AssertionError(f'no edges between {args=} and {onto=}')


def fold(*args, axis: str, direction: str, onto):
    offs = OFFSETS[(axis, direction)]

    if axis not in ('x', 'y', 'z'):
        raise ValueError(f'{axis=} not valid.')    
    if direction == 'cw':
        theta = -m.pi / 2
    elif direction == 'ccw':
        theta = m.pi / 2
    else:
        raise ValueError(f'{direction=} not valid')
    
    A = np.vstack(args)
    off = next(offs)
    reset_origin_tx = edge(A, onto=onto)
    A = translate3d(A, *reset_origin_tx * -1)
    A = rotate(A, axis, theta)
    A = translate3d(A, *reset_origin_tx)
    A = translate3d(A, *off)

    return A



In [1099]:
one = tuple((x, y) for x in range(8, 12) for y in range(4))
two = tuple((x, y) for x in range(4) for y in range(4, 8))
three = tuple((x, y) for x in range(4, 8) for y in range(4, 8))
four = tuple((x, y) for x in range(8, 12) for y in range(4, 8))
five = tuple((x, y) for x in range(8, 12) for y in range(8, 12))
six = tuple((x, y) for x in range(12, 16) for y in range(8, 12))

one = np.array(tuple((x, y, 0) for x in range(50) for y in range(150, 200)))
two = np.array(tuple((x, y, 0) for x in range(50) for y in range(100, 150)))
three = np.array(tuple((x, y, 0) for x in range(50, 100) for y in range(100, 150)))
four = np.array(tuple((x, y, 0) for x in range(50, 100) for y in range(50, 100)))
five = np.array(tuple((x, y, 0) for x in range(50, 100) for y in range(50)))
six = np.array(tuple((x, y, 0) for x in range(100, 150) for y in range(50)))

print(one.shape, two.shape, three.shape, four.shape, five.shape, six.shape)
print(vertices(one))

faces3d = {
    'one': one,
    'two': two,
    'three': three,
    'four': four,
    'five': five,
    'six': six
}

# set them at a height of 0
# faces3d = {}
# for k, v in faces2d.items():
#     faces3d[k] = np.append(v, (np.zeros(16)).reshape(-1, 1), axis=1)


cutout = faces3d
one, two, three, four, five, six = faces3d.values()


(2500, 3) (2500, 3) (2500, 3) (2500, 3) (2500, 3) (2500, 3)
((0, 199, 0), (0, 150, 0), (49, 150, 0), (49, 199, 0))


In [1100]:
plot_3d(faces3d)

In [1166]:

xpos, ypos, zpos = np.array(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
xneg, yneg, zneg = -1 * xpos, -1 * ypos, -1 * zpos

cw_y = itertools.cycle((zpos, xpos, zneg))
ccw_y = itertools.cycle((zpos, xneg, zneg))

cw_x = itertools.cycle((zpos, yneg, zneg))
ccw_x = itertools.cycle((zpos, ypos, zneg))

# no need for first adj b/c already up in the x direction due to first fold
cw_z = itertools.cycle((ypos, xneg))  
ccw_z = itertools.cycle((yneg, xneg))

# thse offsets simulate the wrap-around of flaps with multiple faces
OFFSETS = {
    ('x', 'cw'): cw_x,
    ('x', 'ccw'): ccw_x,
    ('y', 'cw'): cw_y,
    ('y', 'ccw'): ccw_y,
    ('z', 'cw'): cw_z,
    ('z', 'ccw'): ccw_z,    
}

# FOLD UP THE CUBE! !
# f1 = fold(one, onto=four, axis='x', direction='ccw')
# f3, f2 = np.vsplit(fold(three, two, onto=four, axis='y', direction='cw'), 2)
# f2 = fold(f2, onto=f3, axis='y', direction='cw')
# f5, f6 = np.vsplit(fold(five, six, onto=four, axis='x', direction='cw'), 2)
# f6 = fold(f6, onto=f5, axis='z', direction='ccw')

f3, f2, f1 = np.vsplit(fold(three, two, one, onto=four, axis='x', direction='cw'), 3)
f2, f1 = np.vsplit(fold(f2, f1, onto=f3, axis='z', direction='cw'), 2)
f1 = translate3d(f1, 0, -2, 0)
f2 = translate3d(f2, 0, -2, 0)
f1 = fold(f1, onto=f2, axis='y', direction='cw')
f1 = translate3d(f1, 1, 0, -1)
f5, f6 = np.vsplit(fold(five, six, onto=four, axis='x', direction='ccw'), 2)
f6 = fold(f6, onto=f5, axis='z', direction='cw')
f6 = translate3d(f6, 1, 1, 0)



cube = {
    'one': f1,
    'two': f2,
    'three': f3,
    'four': four,
    'five': f5,
    'six': f6,
}


print(vertices(f5))
print(vertices(f6))
# Lets see!
plot_3d(cube)
plot_3d(cutout)



((99.0, 49.0, 50.0), (50.0, 49.0, 1.0), (99.0, 49.0, 1.0), (50.0, 49.0, 50.0))
((100.0, 99.0, 1.0), (100.0, 50.0, 50.0), (100.0, 99.0, 50.0), (100.0, 50.0, 1.0))


In [1075]:
def wrap_around(last_coord, coord, cube=cube) -> tuple[int, int, int]:
    """When we fall off the cube, clip to the nearest point"""
    
    # move to the next coord if it's in the cube
    for k, v in cube.items():
        for point in v:
            if tuple(point) == tuple(coord):
                return coord
    
    # otherwise, find nearest point in cube (should be dist 1 away)
    for k, v in cube.items():
        for point in v:
            if abs(np.linalg.norm(point - coord)) == 1 and tuple(point) != tuple(last_coord):
                return point
            
    raise AssertionError('Fell off the cube!')

            
np.testing.assert_array_equal(wrap_around((8, 4, 0), (8, 3, 0)), np.array((8, 3, 1))) 
np.testing.assert_array_equal(wrap_around((10, 8, 4), (10, 8, 5)), np.array((10, 7, 5))) 


# Good news, each index of a folded face maps to the original point in the 2d face
def jump(coord: tuple[int, ...], to_s: str, cube=cube, cutout=cutout) -> int:
    from_ = cutout if to_s == '3d' else cube
    to_ = cube if to_s == '3d' else cutout
    for k, v in from_.items():
        for i, x in enumerate(v):
            if np.all(x == np.array(coord)):
                return to_[k][i]
    raise AssertionError(f'{coord=} not found in other dimension')


np.testing.assert_array_equal(jump((10, 7, 5), '2d'), np.array((1, 7, 0)))
np.testing.assert_array_equal(jump((1, 7, 0), '3d'), np.array((10, 7, 5)))
np.testing.assert_array_equal(jump((9, 11, 0), '3d'), np.array((9, 8, 4)))
np.testing.assert_array_equal(jump((14, 11, 0), '3d'), np.array((12, 5, 4)))



def move3d(start2d: tuple[int, int, int], dir, cube=cube):  
    start = jump(start2d, '3d')
    
    face = [k for k, v in cube.items() for x in v if tuple(x) == tuple(start)]
    assert len(face) == 1
    face = face[0]
    
    xpos, ypos, zpos = np.array(((1, 0, 0), (0, 1, 0) , (0, 0, 1)))
    xneg, yneg, zneg = np.array(((1, 0, 0), (0, 1, 0) , (0, 0, 1))) * -1
    
    NUDGES = {
        # up, down, left, right
        'one': (zpos, zneg, xneg, xpos),
        'two': (yneg, ypos, xpos, xneg),
        'three': (yneg, ypos, zpos, zneg),
        'four': (yneg, ypos, xneg, xpos),
        'five': (zneg, zpos, xneg, xpos),
        'six': (zneg, zpos, ypos, yneg)
    }
    nu, nd, nl, nr = NUDGES[face]
    
    if dir == 'left':
        nudge = nl
    elif dir == 'right':
        nudge = nr
    elif dir == 'up':
        nudge = nu
    elif dir == 'down':
        nudge = nd
    else:
        raise ValueError(f'{dir=} not valid')

    end = wrap_around(last_coord=start, coord=start+nudge)
    return jump(end, '2d')


np.testing.assert_array_equal(move3d((8, 0, 0), 'down'), np.array((8, 1, 0)))
np.testing.assert_array_equal(move3d((9, 4, 0), 'right'), np.array((10, 4, 0)))
np.testing.assert_array_equal(move3d((11, 5, 0), 'left'), np.array((10, 5, 0)))
np.testing.assert_array_equal(move3d((12, 9, 0), 'up'), np.array((12, 8, 0)))
np.testing.assert_array_equal(move3d((8, 0, 0), 'up'), np.array((3, 4, 0)))
np.testing.assert_array_equal(move3d((10, 11, 0), 'down'), np.array((1, 7, 0)))


    
    
    
    
    


AssertionError: Fell off the cube!

In [949]:
for k, v in cube.items():
        for point in v:
            if tuple(point) == (8, 3, 1):
                print(k, point)

one [8. 3. 1.]


In [898]:
xpos, ypos, zpos = np.array(((1, 0, 0), (0, 1, 0) , (0, 0, 1)))

In [899]:
xpos

array([1, 0, 0])

In [986]:
def find_coord_face(coord: tuple[int, ...], cube) -> str:
    for k, v in cube.items():
        for x in v:
            if tuple(x) == tuple(coord):
                return k
    raise AssertionError('coord not found')

find_coord_face(jump(coord=(8, 0, 0), to_s='3d', cube=cube, cutout=cutout), cube)

'one'

In [ ]:
FACING_FLIPS = {
    # FROM, TO, FACING
    ('one', 'two'): Facing.UP,
    ('one', 'three'): Facing.UP,
    ('one', 'five'): Facing.DOWN,
    ('one', 'six'): Facing.DOWN,
        
    ('two', 'one'): Facing.DOWN,
    ('two', 'three'): Facing.RIGHT,
    ('two', 'four'): Facing.RIGHT,
    ('two', 'five'): Facing.RIG,
    
    ('three', 'one'): Facing.RIGHT,
    ('three', 'two'): Facing.LEFT,
    ('three', 'four'): Facing.RIGHT,
    ('three', 'five'): Facing.RIGHT,
    ('four', 'one'): Facing.UP,
    ('four', 'three'): Facing.LEFT,    
    ('four', 'five'): Facing.DOWN,
    ('four', 'six'): Facing.DOWN,
    ('five', 'two'): Facing.UP,
    ('five', 'three'): Facing.UP,
    ('five', 'four'): Facing.UP,
    ('five', 'six'): Facing.RIGHT,
    ('six', 'one'): Facing.RIGHT,
    ('six', 'two'): Facing.RIGHT,
    ('six', 'four'): Facing.LEFT,
    ('six', 'five'): Facing.LEFT,
}
    

In [1162]:
def generate_radius(coord: tuple[int, int, int], r: int):
    ret = set()
    coord = np.array(coord)
    for x in np.array(list(itertools.product(range(-r, r+1), range(-r, r+1), range(-r, r+1)))):
        if np.linalg.norm((coord + x) - coord) == 1:
            ret.add(tuple(coord + x))
    return ret

In [1165]:
for x in generate_radius((10, 0, 0), 1):
    print(x)

(10, 0, -1)
(10, 0, 1)
(9, 0, 0)
(10, -1, 0)
(10, 1, 0)
(11, 0, 0)


In [1152]:
r = 1
list(itertools.product(range(-r, r+1), range(-r, r+1), range(-r, r+1)))

[(-1, -1, -1),
 (-1, -1, 0),
 (-1, -1, 1),
 (-1, 0, -1),
 (-1, 0, 0),
 (-1, 0, 1),
 (-1, 1, -1),
 (-1, 1, 0),
 (-1, 1, 1),
 (0, -1, -1),
 (0, -1, 0),
 (0, -1, 1),
 (0, 0, -1),
 (0, 0, 0),
 (0, 0, 1),
 (0, 1, -1),
 (0, 1, 0),
 (0, 1, 1),
 (1, -1, -1),
 (1, -1, 0),
 (1, -1, 1),
 (1, 0, -1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, -1),
 (1, 1, 0),
 (1, 1, 1)]

array([50., 98., 51.])